<a href="https://colab.research.google.com/github/vikasgrg/Analytics/blob/main/Data_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd

# Load the dataset
file_path = 'advertising.csv'
df = pd.read_csv(file_path)

# Display basic info and the first few rows
basic_info = df.info()
head = df.head()

df.describe()
df.info()
head

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Daily Time Spent on Site  1000 non-null   float64
 1   Age                       1000 non-null   int64  
 2   Area Income               1000 non-null   float64
 3   Daily Internet Usage      1000 non-null   float64
 4   Ad Topic Line             1000 non-null   object 
 5   City                      1000 non-null   object 
 6   Male                      1000 non-null   int64  
 7   Country                   1000 non-null   object 
 8   Timestamp                 1000 non-null   object 
 9   Clicked on Ad             1000 non-null   int64  
dtypes: float64(3), int64(3), object(4)
memory usage: 78.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  -----

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Male,Country,Timestamp,Clicked on Ad
0,68.95,35,61833.90,256.09,Cloned 5thgeneration orchestration,Wrightburgh,0,Tunisia,27-03-2016 00:53,0
1,80.23,31,68441.85,193.77,Monitored national standardization,West Jodi,1,Nauru,04-04-2016 01:39,0
2,69.47,26,59785.94,236.50,Organic bottom-line service-desk,Davidton,0,San Marino,13-03-2016 20:35,0
3,74.15,29,54806.18,245.89,Triple-buffered reciprocal time-frame,West Terrifurt,1,Italy,10-01-2016 02:31,0
4,68.37,35,73889.99,225.58,Robust logistical utilization,South Manuel,0,Iceland,03-06-2016 03:36,0


In [6]:
from sklearn.model_selection import train_test_split

# Select features (exclude non-numeric / ID-like columns)
X = df[['Daily Time Spent on Site', 'Age', 'Area Income', 'Daily Internet Usage', 'Male']]

# Target variable
y = df['Clicked on Ad']

# Split the data (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Shapes of the splits
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((700, 5), (300, 5), (700,), (300,))

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

#==================================================================================
# Stratified sampling -cross validation
#==================================================================================

# Initialize and fit the logistic regression model
# log_reg = LogisticRegression()

#Originally the code was not having max_iter parameter
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

# prompt: calculate the training and test accuracy

# Make predictions on the training and test sets
y_train_pred = log_reg.predict(X_train)
y_test_pred = log_reg.predict(X_test)

# Calculate accuracy scores
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Print the accuracy scores
print(f"Training Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")



Training Accuracy: 0.9685714285714285
Test Accuracy: 0.9766666666666667


In [ ]:
#==================================================================================
# shuffle split cross validation
#==================================================================================


from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear')

# Set up ShuffleSplit cross-validator
from sklearn.model_selection import ShuffleSplit, cross_validate
shuffle_split = ShuffleSplit(n_splits=200, test_size=0.3, random_state=42)

# Perform cross-validation and collect both train and test scores
cv_results = cross_validate(model, X, y, cv=shuffle_split, scoring='accuracy', return_train_score=True)

# Extract train and test scores
train_scores = cv_results['train_score']
test_scores = cv_results['test_score']

# Show individual scores and their means
print("cross validation: test accuracy:" , np.round(train_scores.mean(),2))
print("cross validation: test accuracy:" , np.round(test_scores.mean(),2))


cross validation: test accuracy: 0.9
cross validation: test accuracy: 0.9


In [ ]:
#=============================================================================
# K-Fold cross validation
#=============================================================================

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear')

# Set up ShuffleSplit cross-validator
from sklearn.model_selection import KFold, cross_validate
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation and collect both train and test scores
cv_results = cross_validate(model, X, y, cv=shuffle_split, scoring='accuracy', return_train_score=True)

# Extract train and test scores
train_scores = cv_results['train_score']
test_scores = cv_results['test_score']

# Show individual scores and their means
print("cross validation: train accuracy:" , np.round(train_scores.mean(),2))
print("cross validation: test accuracy:" , np.round(test_scores.mean(),2))


cross validation: test accuracy: 0.9
cross validation: test accuracy: 0.9


In [ ]:
#=============================================================================
# Stratified Splitting
#=============================================================================

from sklearn.model_selection import StratifiedShuffleSplit

# Stratified Split to maintain class proportions
strat_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Extract the train and test indices
for train_index, test_index in strat_split.split(X, y):
    X_train_strat, X_test_strat = X.iloc[train_index], X.iloc[test_index]
    y_train_strat, y_test_strat = y.iloc[train_index], y.iloc[test_index]

# Scale the features
#X_train_strat_scaled = scaler.fit_transform(X_train_strat)
#X_test_strat_scaled = scaler.transform(X_test_strat)

# Train logistic regression on stratified data
log_reg_strat = LogisticRegression()
log_reg_strat.fit(X_train_strat, y_train_strat)

# Predict
y_train_strat_pred = log_reg_strat.predict(X_train_strat)
y_test_strat_pred = log_reg_strat.predict(X_test_strat)

# Accuracy
train_strat_accuracy = accuracy_score(y_train_strat, y_train_strat_pred)
test_strat_accuracy = accuracy_score(y_test_strat, y_test_strat_pred)

train_strat_accuracy, test_strat_accuracy
print("train_strat_accuracy",np.round(np.mean(train_strat_accuracy),2))
print("test_strat_accuracy",np.round(np.mean(test_strat_accuracy),2))

train_strat_accuracy 0.93
test_strat_accuracy 0.97


C:\Users\excel\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
